In [ ]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/src")
from importlib import reload
import utils
reload(utils)
from utils import *
data_folder = "/nfs/home/genovese/thesis-wildfire-genovese/data/"
output_folder = "/nfs/home/genovese/thesis-wildfire-genovese/data/gathering/per_comune/"

In [ ]:
ist = pd.read_excel('/nfs/home/genovese/thesis-wildfire-genovese/data/raw/istruzione_occupazione/istruzione_2018-2022.xlsx')
occ = pd.read_excel('/nfs/home/genovese/thesis-wildfire-genovese/data/raw/istruzione_occupazione/occupazione_2018_2022.xlsx')

In [ ]:
confini = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/clean_data/confini_piemonte/confini_piemonte.shp')

---

In [ ]:
istruzione = dict.fromkeys([2018, 2019, 2020, 2021, 2022])
for i, a in enumerate(istruzione.keys()):
    istruzione[a] = ist.iloc[:-1, [0]+list(range(1+(11*i), 12+(11*i)))]

In [ ]:
for a, df in istruzione.items():
    df.columns = df.iloc[0, :]
    df.drop(index=df.index[:2], inplace=True)

Remove data gathered at region/province level

In [ ]:
for a, df in istruzione.items():
    to_drop = list(df[df['Grado di istruzione'].str.strip().duplicated(keep='last')].index) + list(df[df['Grado di istruzione'].str.strip() == 'Piemonte'].index)
    istruzione[a] = df.drop(index = to_drop).reset_index(drop=True)

In [ ]:
for a,df in istruzione.items():
    aux = df.apply(lambda x: uniform_strings(x, 'Grado di istruzione'), axis=1).replace('..', -1000).fillna(-1000)
    for c in aux.columns[1:]:
        aux.loc[:, c] = aux[c].astype('int64')
        aux.loc[:, c] = aux[c].replace(-1000, None)
    istruzione[a] = aux

In [ ]:
for a, df in istruzione.items():
    istruzione[a] = confini[['comune', 'geometry']].merge(df.rename(columns={'Grado di istruzione': 'comune'}), on='comune', how='inner')

In [ ]:
for a, df in istruzione.items():    
    save_clean_data(df.drop(columns='comune'), str(a), '/nfs/home/genovese/thesis-wildfire-genovese/data/gathering/per_comune/istruzione')

---

In [ ]:
occupazione = dict.fromkeys([2018, 2019, 2020, 2021, 2022])
for i, a in enumerate(occupazione.keys()):
    occupazione[a] = occ.iloc[1:-1, [0]+list(range(1+(9*i), 10+(9*i)))]

In [ ]:
for a, df in occupazione.items():
    df.columns = df.iloc[0, :]
    df.drop(index=df.index[:2], inplace=True)

In [ ]:
for a, df in occupazione.items():
    to_drop = list(df[df['Condizione professionale'].str.strip().duplicated(keep='last')].index) + list(df[df['Condizione professionale'].str.strip() == 'Piemonte'].index)
    occupazione[a] = df.drop(index = to_drop).reset_index(drop=True)

In [ ]:
for a,df in occupazione.items():
    aux = df.apply(lambda x: uniform_strings(x, 'Condizione professionale'), axis=1).replace('..', -1000).fillna(-1000)
    for c in aux.columns[1:]:
        aux.loc[:, c] = aux[c].astype('int64')
        aux.loc[:, c] = aux[c].replace(-1000, None)
    occupazione[a] = aux

In [ ]:
for a, df in occupazione.items():
    occupazione[a] = confini[['comune', 'geometry']].merge(df.rename(columns={'Condizione professionale': 'comune'}), on='comune', how='inner')

In [ ]:
for a, df in occupazione.items():    
    save_clean_data(df.drop(columns='comune'), str(a), '/nfs/home/genovese/thesis-wildfire-genovese/data/gathering/per_comune/occupazione')